In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from functools import partial

from shrun import sync

# Code

In [ ]:
def init_sftp_args():
    sftp_args = {
        'password': os.environ['SFTPPASS'],
        'username': os.environ['SFTPUSER'],
    }
    return sftp_args

def sftp(password, username, src, dst, action):
    import paramiko
    if action == 'push': host, port, dst = dst.split(':', 2)
    else: host, port, src = src.split(':', 2)
    
    transport = paramiko.Transport((host, int(port)))
    transport.connect(username = username, password = password)
    sftp = paramiko.SFTPClient.from_transport(transport)
    
    if action == 'pull': sftp.get(src, dst)
    elif action == 'push': sftp.put(src, dst)
    
    sftp.close()
    transport.close()

pusher = partial(sftp, action='push')
puller = partial(sftp, action='pull')

Syncer = sync.SyncerBase(sender=pusher, reciever=puller, kwargs=init_sftp_args())